In [88]:
import glob
import pandas as pd
import numpy as np

import os
import re

In [89]:
attack = "crownibp"

In [90]:
results_folder = rf"results\{attack}"

In [91]:
algorithms = ["baseline","noise","fgsm","pgd","crownibp"]

In [116]:
# CI , IBS , NegLL
metric = "NegLL"

In [117]:
ascending = False if metric=="CI" else True

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{metric}.xlsx")
    excels.extend(glob.glob(glob_search))

In [118]:
df = pd.DataFrame()
N_datasets = len(excels)
percentage_change = []
print(excels)
for excel_i in excels:

    temp_df = pd.read_excel(excel_i)
    folder_name,dataset_name = excel_i.split("\\")[-3:-1]
    temp_df.columns = ["eps"] + temp_df.columns[1:].to_list()
    col_name = (dataset_name,re.sub("results_","",folder_name))
    df[col_name] = temp_df[f"Robust {metric}"] #temp_df["Non Robust CI"].round(3).astype(str) + " / " + temp_df["Robust CI"].round(3).astype(str)

    col_name = (dataset_name,"baseline")
    df[col_name] = temp_df[f"Non Robust {metric}"]

df.index = temp_df.eps

['results\\crownibp\\results_crownibp\\Aids2\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\dataDIVAT1\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\Dialysis\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\divorce\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\flchain\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\Framingham\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\Pbc3\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\prostate\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\retinopathy\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\stagec\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\TRACE\\NegLL.xlsx', 'results\\crownibp\\results_crownibp\\vlbw\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\Aids2\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\dataDIVAT1\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\Dialysis\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\divorce\\NegLL.xlsx', 'results\\crownibp\\results_fgsm\\flchain\\NegLL.xls

In [119]:
micolumns = pd.MultiIndex.from_tuples(df.columns)
dfmi = (
    pd.DataFrame(
        df.values,
        index=df.index,
        columns=micolumns,
    )
    .sort_index()
    .sort_index(axis=1)
)

dfmi.sort_values(by="eps",ascending=False,inplace=True)
dfmi = dfmi.reindex(columns=algorithms, level=1)


In [120]:
dfmi[dfmi==''] = np.NaN
dfmi = dfmi.astype(float)
dfmi

Aids2                                                         \
           baseline         noise          fgsm           pgd     crownibp   
eps                                                                          
2.00  867218.625000  1.664442e+09  5.247136e+07  4.170300e+07  1013.211182   
1.00   11408.425781  3.843492e+05  6.768741e+04  5.493030e+04   529.279114   
0.80    4595.645508  6.678125e+04  1.776805e+04  1.489971e+04   522.198975   
0.70    2868.485596  2.779730e+04  8.894230e+03  7.577893e+03   521.521118   
0.60    1798.874268  1.135636e+04  4.351412e+03  3.794776e+03   521.171753   
0.50    1159.978516  4.424974e+03  2.142487e+03  1.915283e+03   520.945190   
0.10     515.993896  5.196252e+02  5.187775e+02  5.186812e+02   521.655640   
0.07     516.228821  5.188468e+02  5.188582e+02  5.188345e+02   521.751038   
0.05     516.784729  5.191172e+02  5.191102e+02  5.190440e+02   521.819092   
0.00     518.514893  5.208565e+02  5.195742e+02  5.194845e+02   521.989746   

          Dialysis                                                          \
          baseline         noise          fgsm           pgd      crownibp   
eps                                                                          
2.00  4.559872e+16           NaN           NaN  2.248880e+33  29887.589844   
1.00  9.240066e+08  6.920299e+34  7.336558e+19  3.604828e+16   2584.852295   
0.80  3.504494e+07  2.610555e+27  8.823269e+15  2.004266e+13   1959.104614   
0.70  7.014061e+06  5.569553e+23  9.986558e+13  4.851810e+11   1776.846558   
0.60  1.445596e+06  1.300917e+20  1.149362e+12  1.224906e+10   1658.694092   
0.50  3.095362e+05  3.559136e+16  1.365481e+10  3.267395e+08   1591.970093   
0.10  1.812542e+03  5.482956e+03  1.955184e+03  1.672969e+03   1523.746338   
0.07  1.644429e+03  2.364513e+03  1.608519e+03  1.546026e+03   1523.031616   
0.05  1.579474e+03  1.844645e+03  1.542314e+03  1.521537e+03   1522.824341   
0.00  1.516727e+03  1.567583e+03  1.521137e+03  1.514367e+03   1523.203857   

      ...        stagec                                            \
      ...      baseline         noise          fgsm           pgd   
eps   ...                                                           
2.00  ...  1.491603e+12  5.955046e+12  4.591150e+10  3.632862e+10   
1.00  ...  1.161569e+06  2.070726e+06  1.796648e+05  1.538292e+05   
0.80  ...  7.496040e+04  1.143366e+05  1.672986e+04  1.469834e+04   
0.70  ...  1.853756e+04  2.733591e+04  5.226173e+03  4.627388e+03   
0.60  ...  4.810933e+03  6.494227e+03  1.638094e+03  1.474163e+03   
0.50  ...  1.282302e+03  1.608063e+03  5.300793e+02  4.837622e+02   
0.10  ...  4.104847e+01  4.103425e+01  3.957289e+01  3.934553e+01   
0.07  ...  4.007188e+01  3.998076e+01  3.916440e+01  3.899010e+01   
0.05  ...  3.975593e+01  3.960278e+01  3.907390e+01  3.891127e+01   
0.00  ...  3.966661e+01  3.950724e+01  3.916957e+01  3.904496e+01   

                             vlbw                                            \
           crownibp      baseline         noise          fgsm           pgd   
eps                                                                           
2.00  580507.312500  9.966138e+17           NaN  1.678173e+28  9.349412e+27   
1.00    1032.293945  6.365167e+08  8.356319e+16  2.364262e+13  7.933361e+12   
0.80     330.232391  1.405074e+07  9.150955e+12  4.327570e+10  1.458611e+10   
0.70     194.125397  2.233256e+06  9.764940e+10  1.882171e+09  6.370283e+08   
0.60     120.920013  3.588179e+05  1.846748e+09  8.237950e+07  2.990652e+07   
0.50      82.148254  5.837540e+04  4.295040e+07  3.689856e+06  1.422935e+06   
0.10      38.943516  1.165087e+02  1.594256e+02  1.210428e+02  1.016980e+02   
0.07      38.505737  8.887656e+01  9.697730e+01  8.381541e+01  7.358942e+01   
0.05      38.301704  7.527724e+01  7.223357e+01  6.864921e+01  6.115559e+01   
0.00      38.086372  5.109546e+01  4.132962e+01  4.518285e+01  4.120808e+01   

                    
          crownibp  
eps             

In [121]:
# dfmi.T.groupby(axis=0,level=0).rank(axis=0,na_option='bottom',method="average",ascending=ascending)

In [122]:
best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=0,na_option='bottom',method="average",ascending=ascending)
best_per_dataset_rank

C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\996602392.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=0,na_option='bottom',method="average",ascending=ascending)
C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\996602392.py:1: FutureWarning: The 'axis' keyword in DataFrameGroupBy.rank is deprecated and will be removed in a future version. Call without passing 'axis' instead.
  best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=0,na_option='bottom',method="average",ascending=ascending)


Aids2                          Dialysis                           ...  \
     baseline noise fgsm  pgd crownibp baseline noise fgsm  pgd crownibp  ...   
eps                                                                       ...   
2.00      2.0   5.0  4.0  3.0      1.0      2.0   4.5  4.5  3.0      1.0  ...   
1.00      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.80      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.70      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.60      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.50      2.0   5.0  4.0  3.0      1.0      2.0   5.0  4.0  3.0      1.0  ...   
0.10      1.0   4.0  3.0  2.0      5.0      3.0   5.0  4.0  2.0      1.0  ...   
0.07      1.0   3.0  4.0  2.0      5.0      4.0   5.0  3.0  2.0      1.0  ...   
0.05      1.0   4.0  3.0  2.0      5.0      4.0   5.0  3.0  1.0      2.0  ...   
0.00      1.0   4.0  3.0  2.0      5.0      2.0   5.0  3.0  1.0      4.0  ...   

       stagec                              vlbw                           
     baseline noise fgsm  pgd crownibp baseline noise fgsm  pgd crownibp  
eps                                                                       
2.00      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
1.00      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.80      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.70      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.60      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.50      4.0   5.0  3.0  2.0      1.0      2.0   5.0  4.0  3.0      1.0  
0.10      5.0   4.0  3.0  2.0      1.0      3.0   5.0  4.0  1.0      2.0  
0.07      5.0   4.0  3.0  2.0      1.0      3.0   5.0  2.0  1.0      4.0  
0.05      5.0   4.0  3.0  2.0      1.0      4.0   3.0  2.0  1.0      5.0  
0.00      5.0   4.0  3.0  2.0      1.0      4.0   2.0  3.0  1.0      5.0  

[10 rows x 60 columns]

In [123]:
best_per_dataset_avg_rank = best_per_dataset_rank.stack(level=1).mean(1).unstack(1).sort_values(by="eps",ascending=False)
best_per_dataset_avg_rank

,baseline,noise,fgsm,pgd,crownibp
eps,,,,,
2.00,3.000000,4.208333,3.708333,3.083333,1.000000
1.00,3.000000,4.416667,3.583333,3.000000,1.000000
0.80,2.916667,4.500000,3.583333,3.000000,1.000000
0.70,3.000000,4.500000,3.500000,3.000000,1.000000
0.60,2.916667,4.666667,3.500000,2.916667,1.000000
0.50,2.916667,4.666667,3.500000,2.916667,1.000000
0.10,3.333333,4.583333,3.000000,2.166667,1.916667
0.07,3.416667,4.166667,2.916667,2.250000,2.250000
0.05,3.416667,4.000000,2.750000,2.250000,2.583333


In [124]:
if not ascending:
    best_per_dataset = dfmi.groupby(level=0,axis=1).idxmax(1)
else:
    best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
best_per_dataset.applymap(lambda x: x[1])

C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\1722459016.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\1722459016.py:4: FutureWarning: DataFrameGroupBy.idxmin with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\1722459016.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1])


,Aids2,Dialysis,Framingham,Pbc3,TRACE,dataDIVAT1,divorce,flchain,prostate,retinopathy,stagec,vlbw
eps,,,,,,,,,,,,
2.00,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
1.00,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.80,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.70,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.60,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.50,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.10,baseline,crownibp,crownibp,baseline,crownibp,crownibp,fgsm,crownibp,crownibp,pgd,crownibp,pgd
0.07,baseline,crownibp,crownibp,noise,crownibp,crownibp,fgsm,crownibp,crownibp,pgd,crownibp,pgd
0.05,baseline,pgd,crownibp,noise,crownibp,baseline,fgsm,crownibp,crownibp,fgsm,crownibp,pgd


In [125]:
if metric != "NegLL":
     # aggregate all the CI files
    os.listdir(results_folder)
    excels = []
    for folder in os.listdir(results_folder):
        glob_search = os.path.join(results_folder,folder,f"{metric}_all.xlsx")
        excels.extend(glob.glob(glob_search))
        
    df = pd.DataFrame()
    N_datasets = len(excels)
    percentage_change = []
    for excel_i in excels:
    
        temp_df = pd.read_excel(excel_i)
        folder_name = excel_i.split("\\")[-2]
        temp_df.columns = ["eps"] + temp_df.columns[1:].to_list()
        col_name = re.sub("results_","",folder_name)
        df[col_name] = temp_df["%"] #temp_df["Non Robust CI"].round(3).astype(str) + " / " + temp_df["Robust CI"].round(3).astype(str)
    
    df.index = temp_df.eps
    df = df.reindex(columns=algorithms[1:], level=1)

In [126]:
df

,"(Aids2, crownibp)","(Aids2, baseline)","(dataDIVAT1, crownibp)","(dataDIVAT1, baseline)","(Dialysis, crownibp)","(Dialysis, baseline)","(divorce, crownibp)","(divorce, baseline)","(flchain, crownibp)","(flchain, baseline)",...,"(Dialysis, pgd)","(divorce, pgd)","(flchain, pgd)","(Framingham, pgd)","(Pbc3, pgd)","(prostate, pgd)","(retinopathy, pgd)","(stagec, pgd)","(TRACE, pgd)","(vlbw, pgd)"
eps,,,,,,,,,,,,,,,,,,,,,
2.00,1013.211182,867218.625000,20130.279297,3.917180e+06,29887.589844,4.559872e+16,1093.994751,1.627015e+07,1.400884e+12,NaN,...,2.248880e+33,740785.625000,inf,1.258766e+14,5.015566e+11,1.691516e+26,6.762288e+06,3.632862e+10,8.507489e+18,9.349412e+27
1.00,529.279114,11408.425781,918.089050,1.433132e+04,2584.852295,9.240066e+08,1051.740601,2.363391e+04,6.307257e+03,3.350622e+23,...,3.604828e+16,4027.023438,1.304035e+16,2.014873e+07,4.558995e+05,8.837224e+12,7.303170e+03,1.538292e+05,4.530066e+08,7.933361e+12
0.80,522.198975,4595.645508,821.152588,4.946042e+03,1959.104614,3.504494e+07,1048.827271,6.934156e+03,1.425336e+03,2.564323e+18,...,2.004266e+13,1892.923462,4.929555e+11,9.573399e+05,3.244131e+04,2.175150e+10,2.043456e+03,1.469834e+04,5.348799e+06,1.458611e+10
0.70,521.521118,2868.485596,800.327820,3.008757e+03,1776.846558,7.014061e+06,1047.909302,4.025990e+03,1.194910e+03,7.372112e+15,...,4.851810e+11,1468.120728,3.470649e+09,2.129752e+05,9.031110e+03,1.136398e+09,1.108950e+03,4.627388e+03,6.115734e+05,6.370283e+08
0.60,521.171753,1798.874268,785.855286,1.914498e+03,1658.694092,1.445596e+06,1047.239380,2.521232e+03,1.115522e+03,2.230515e+13,...,1.224906e+10,1233.591675,4.597551e+07,4.876955e+04,2.552643e+03,5.942480e+07,6.254221e+02,1.474163e+03,7.626407e+04,2.990652e+07
0.50,520.945190,1159.978516,774.627319,1.311128e+03,1591.970093,3.095362e+05,1046.801880,1.788535e+03,1.089056e+03,6.824825e+10,...,3.267395e+08,1116.159424,1.331019e+06,1.211749e+04,7.240753e+02,3.266468e+06,3.810739e+02,4.837622e+02,1.071335e+04,1.422935e+06
0.10,521.655640,515.993896,751.842834,7.562355e+02,1523.746338,1.812542e+03,1045.666992,1.056980e+03,1.069681e+03,2.033814e+03,...,1.672969e+03,1044.659546,1.102356e+03,1.499732e+03,3.124055e+01,6.400184e+02,1.682910e+02,3.934553e+01,5.403820e+02,1.016980e+02
0.07,521.751038,516.228821,751.301270,7.522886e+02,1523.031616,1.644429e+03,1045.636963,1.050426e+03,1.069362e+03,1.443374e+03,...,1.546026e+03,1044.730347,1.084690e+03,1.495175e+03,3.113774e+01,5.438411e+02,1.674806e+02,3.899010e+01,5.340093e+02,7.358942e+01
0.05,521.819092,516.784729,751.017822,7.505762e+02,1522.824341,1.579474e+03,1045.620850,1.047734e+03,1.069202e+03,1.250709e+03,...,1.521537e+03,1044.790894,1.078688e+03,1.493774e+03,3.125700e+01,5.001221e+02,1.672798e+02,3.891127e+01,5.309935e+02,6.115559e+01


In [114]:
excel_name = os.path.join(results_folder,f"{metric}_all.xlsx")
with pd.ExcelWriter(excel_name) as writer:  
    dfmi.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
    best_per_dataset_rank.to_excel(writer,sheet_name="rank")
    best_per_dataset_avg_rank.to_excel(writer,sheet_name="average_rank")
    best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")
    df.to_excel(writer,sheet_name="%")

C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\229516633.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfmi.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\229516633.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")


In [115]:
best_per_dataset_avg_rank.index = ["{:.2f}".format(float(x)) for x in np.round(best_per_dataset_avg_rank.index.tolist(),2)]
best_per_dataset_avg_rank.index.name = r"$\epsilon$"
print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))

\begin{tabular}{llllll}
\toprule
 & baseline & noise & fgsm & pgd & crownibp \\
$\epsilon$ &  &  &  &  &  \\
\midrule
2.00 & 3.04 & 4.0 & 3.5 & 3.33 & 1.12 \\
1.00 & 3.12 & 3.92 & 3.75 & 3.21 & 1.0 \\
0.80 & 3.04 & 4.21 & 3.54 & 3.21 & 1.0 \\
0.70 & 2.92 & 4.33 & 3.5 & 3.25 & 1.0 \\
0.60 & 3.0 & 4.38 & 3.54 & 3.08 & 1.0 \\
0.50 & 3.25 & 4.42 & 3.33 & 3.0 & 1.0 \\
0.10 & 3.25 & 4.25 & 3.17 & 2.42 & 1.92 \\
0.07 & 3.33 & 3.92 & 3.17 & 2.5 & 2.08 \\
0.05 & 3.17 & 4.08 & 3.0 & 2.58 & 2.17 \\
0.00 & 3.0 & 3.75 & 3.33 & 2.5 & 2.42 \\
\bottomrule
\end{tabular}



C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\3581541270.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))


In [44]:
if metric == "NegLL":
    dfmi.index = ["{:.2f}".format(float(x)) for x in np.round(dfmi.index.tolist(),2)]
    dfmi.index.name = r"$\epsilon$"
    print(dfmi.applymap(lambda x: np.round(x,3)).applymap(lambda x: "{:.2e}".format(x)).to_latex(index=True,multicolumn_format="c"))
else:
    dfmi.index = ["{:.2f}".format(float(x)) for x in np.round(dfmi.index.tolist(),2)]
    dfmi.index.name = r"$\epsilon$"
    print(dfmi.applymap(lambda x: np.round(x,3)).applymap(str).to_latex(index=True,multicolumn_format="c"))

\begin{tabular}{lllllllllllllllllllllllllllllllllllllllll}
\toprule
 & \multicolumn{5}{c}{Aids2} & \multicolumn{5}{c}{Dialysis} & \multicolumn{5}{c}{Framingham} & \multicolumn{5}{c}{TRACE} & \multicolumn{5}{c}{dataDIVAT1} & \multicolumn{5}{c}{divorce} & \multicolumn{5}{c}{flchain} & \multicolumn{5}{c}{prostate} \\
 & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp \\
$\epsilon$ &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
2.00 & 8.67e+05 & 1.66e+09 & 5.25e+07 & 4.17e+07 & 1.01e+03 & 4.56e+16 & nan & nan & 2.25e+33 & 2.99e+04 & 2.77e+08 & 2.34e+13 & 3.64e+11 & 1.26e+14 & 3.17e+03 & 9.02e+19 & 2.25e+18 & 3.63e+25 

C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\718377934.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(dfmi.applymap(lambda x: np.round(x,3)).applymap(lambda x: "{:.2e}".format(x)).to_latex(index=True,multicolumn_format="c"))


In [45]:
# pd.set_option('display.max_columns', None)
# print(dfmi)

In [46]:
print(best_per_dataset.applymap(lambda x: x[1]).to_latex(index=False))

\begin{tabular}{llllllll}
\toprule
Aids2 & Dialysis & Framingham & TRACE & dataDIVAT1 & divorce & flchain & prostate \\
\midrule
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
baseline & crownibp & crownibp & crownibp & crownibp & fgsm & crownibp & crownibp \\
baseline & crownibp & crownibp & crownibp & crownibp & fgsm & crownibp & crownibp \\
baseline & pgd & crownibp & crownibp & baseline & fgsm & crownibp & crownibp \\
baseline & pgd & crownibp & pgd & baseline & fgsm & crownibp & crownibp \\
\bottomrule


C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\3574660769.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset.applymap(lambda x: x[1]).to_latex(index=False))


In [47]:
df.index = ["{:.2f}".format(float(x)) for x in np.round(df.index.tolist(),2)]
df.index.name = r"$\epsilon$"
print(df.applymap(lambda x: str(np.round(x,3))).to_latex(index=True))

\begin{tabular}{lllllllllllllllllllllllllllllllllllllllll}
\toprule
 & ('Aids2', 'crownibp') & ('Aids2', 'baseline') & ('dataDIVAT1', 'crownibp') & ('dataDIVAT1', 'baseline') & ('Dialysis', 'crownibp') & ('Dialysis', 'baseline') & ('divorce', 'crownibp') & ('divorce', 'baseline') & ('flchain', 'crownibp') & ('flchain', 'baseline') & ('Framingham', 'crownibp') & ('Framingham', 'baseline') & ('prostate', 'crownibp') & ('prostate', 'baseline') & ('TRACE', 'crownibp') & ('TRACE', 'baseline') & ('Aids2', 'fgsm') & ('dataDIVAT1', 'fgsm') & ('Dialysis', 'fgsm') & ('divorce', 'fgsm') & ('flchain', 'fgsm') & ('Framingham', 'fgsm') & ('prostate', 'fgsm') & ('TRACE', 'fgsm') & ('Aids2', 'noise') & ('dataDIVAT1', 'noise') & ('Dialysis', 'noise') & ('divorce', 'noise') & ('flchain', 'noise') & ('Framingham', 'noise') & ('prostate', 'noise') & ('TRACE', 'noise') & ('Aids2', 'pgd') & ('dataDIVAT1', 'pgd') & ('Dialysis', 'pgd') & ('divorce', 'pgd') & ('flchain', 'pgd') & ('Framingham', 'pgd') & ('pros

C:\Users\lpott\AppData\Local\Temp\ipykernel_22436\3740848601.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df.applymap(lambda x: str(np.round(x,3))).to_latex(index=True))


## PDF PICTURES

In [48]:
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import pandas as pd
import re

In [49]:
attack = "crownibp"

In [50]:
results_folder = rf"results\{attack}"

In [51]:
algorithms = ["baseline","noise","fgsm","pgd","crownibp"]

In [52]:
# CI , IBS , NegLL
img_name = "population_curves_attacked_test"

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{img_name}.xlsx")
    excels.extend(glob.glob(glob_search))

In [53]:
metadata = 'eps'

In [54]:
n_rows = int(len(excels) / (len(algorithms)-1))
n_cols = int(len(algorithms) - 1)

In [ ]:
excels = np.sort(excels)
files = pd.DataFrame(excels.reshape(-1,len(algorithms)-1,order="F"),columns=["crownibp","fgsm","noise","pgd"]).reindex(["noise","fgsm","pgd","crownibp"],axis=1).values

In [ ]:
df_population = pd.read_excel(excels[0])

In [ ]:
df_population

In [ ]:
save_folder = results_folder = os.path.join(fr"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results\{attack}","perturb_curves.pdf")

In [ ]:
print(save_folder)

In [ ]:
save_folder = results_folder = os.path.join(r"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results",attack,"perturb_curves.pdf")

fig,axes = plt.subplots(n_rows,n_cols+1,figsize=(30,40),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(n_rows):
    for j in range(n_cols):
        print(i,j)
        fileij = files[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        df_population= df_population.iloc[:,3:]

        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in robust_df.columns]
        baseline_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in baseline_df.columns]
        
        base1 = axes[i][j+1].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
        base2 = axes[i][j+1].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
        base3 = axes[i][j+1].plot(t,base_models.iloc[:,2],linewidth=3,c="k")

  
        robust1 = axes[i][j+1].plot(t,robust_df,'--',linewidth=3)
  
        if j == 0:
            base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
            base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
            base3 = axes[i][j].plot(t,base_models.iloc[:,2],linewidth=3,c="k")
            baseline1 = axes[i][j].plot(t,baseline_df,'--',linewidth=3)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)

        axes[i][j+1].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = base_models.columns.tolist() + robust_df.columns.tolist()
fig.legend([base1, base2,base3,robust1], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.06)) 

# axes[0][3].legend(base_models.columns.tolist() + robust_df.columns.tolist(),fontsize=20,ncol=2,loc=1)

# plt.legend(base_models.columns.tolist() + robust_df.columns.tolist(),loc='upper center',ncol=5)
plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

In [ ]:
files[2:4,:]

In [ ]:
save_folder = results_folder = os.path.join(r"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results",attack,"perturb_curves_subset.pdf")
file_subset = files[2:4,:]

fig,axes = plt.subplots(2,n_cols+1,figsize=(30,12),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(2):
    for j in range(n_cols):
        print(i,j)
        fileij = file_subset[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        if dataset not in ["TRACE","Framingham"]:
            continue
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        df_population= df_population.iloc[:,3:]

        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in robust_df.columns]
        baseline_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in baseline_df.columns]
        
        base1 = axes[i][j+1].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
        base2 = axes[i][j+1].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
        base3 = axes[i][j+1].plot(t,base_models.iloc[:,2],linewidth=3,c="k")

  
        robust1 = axes[i][j+1].plot(t,robust_df,'--',linewidth=3)
  
        if j == 0:
            base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
            base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
            base3 = axes[i][j].plot(t,base_models.iloc[:,2],linewidth=3,c="k")
            baseline1 = axes[i][j].plot(t,baseline_df,'--',linewidth=3)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)

        axes[i][j+1].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = base_models.columns.tolist() + robust_df.columns.tolist()
fig.legend([base1, base2,base3,robust1], labels=labels, 
           loc="upper center",ncols=6,fontsize=30,bbox_to_anchor=(.5,1.15)) 

# axes[0][3].legend(base_models.columns.tolist() + robust_df.columns.tolist(),fontsize=20,ncol=2,loc=1)

# plt.legend(base_models.columns.tolist() + robust_df.columns.tolist(),loc='upper center',ncol=5)
plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

### DIST PLOTS

In [ ]:
# CI , IBS , NegLL
results_folder = fr"results\{attack}"
img_name = "curve_distributions_test"

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{img_name}.xlsx")
    excels.extend(glob.glob(glob_search))

In [ ]:
excels = np.sort(excels)
files = pd.DataFrame(excels.reshape(-1,len(algorithms)-1,order="F"),columns=["crownibp","fgsm","noise","pgd"]).reindex(["noise","fgsm","pgd","crownibp"],axis=1).values

In [ ]:
save_folder = results_folder = os.path.join(r"results",attack,"dist_curves.pdf")

In [ ]:
import seaborn as sns

In [ ]:
fig,axes = plt.subplots(n_rows,n_cols+1,figsize=(30,40),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(n_rows):
    for j in range(n_cols):
        print(i,j)
        fileij = files[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = [col.split("_")[1] for col in robust_df.columns]
        baseline_df.columns =[col.split("_")[1] for col in baseline_df.columns]
        
        mu = sns.lineplot(x=t, y=robust_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j+1],c='b',legend=False)
        q95 = sns.lineplot(x=t, y=robust_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        q05 = sns.lineplot(x=t, y=robust_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)

        line = q05.get_lines()
        axes[i][j+1].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
        axes[i][j+1].set_xlabel("t",fontsize=20)

        if j == 0:
            mu = sns.lineplot(x=t, y=baseline_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j],c='b',legend=False)
            q95 = sns.lineplot(x=t, y=baseline_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)
            q05 = sns.lineplot(x=t, y=baseline_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)
            line = q05.get_lines()
            axes[i][j].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
            axes[i][j].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = ["S(t)","Credible Interval","$Q_{95},Q_{05}$"]
fig.legend([mu,q95,q05], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.03)) 

plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

In [ ]:
save_folder = results_folder = os.path.join(r"results",attack,"dist_curves_subset.pdf")
file_subset = files[2:4,:]

fig,axes = plt.subplots(2,n_cols+1,figsize=(30,12),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(2):
    for j in range(n_cols):
        print(i,j)
        fileij = file_subset[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = [col.split("_")[1] for col in robust_df.columns]
        baseline_df.columns =[col.split("_")[1] for col in baseline_df.columns]
        
        mu = sns.lineplot(x=t, y=robust_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j+1],c='b',legend=False)
        q95 = sns.lineplot(x=t, y=robust_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        q05 = sns.lineplot(x=t, y=robust_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        axes[i][j+1].set_xlabel("t",fontsize=20)

        line = q05.get_lines()
        axes[i][j+1].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
        if j == 0:
            mu = sns.lineplot(x=t, y=baseline_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j],c='b',legend=False)
            q95 = sns.lineplot(x=t, y=baseline_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)
            q05 = sns.lineplot(x=t, y=baseline_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)
            line = q05.get_lines()
            axes[i][j].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
            axes[i][j].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = ["S(t)","Credible Interval","$Q_{95},Q_{05}$"]
fig.legend([mu,q95,q05], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.12)) 

plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()